In [1]:
import pandas as pd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString

In [2]:
traj_ori = pd.read_csv('../data/traj.csv')
traj_ori

,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays
0,0,2013-10-08T17:45:00Z,254,0,"[116.318726,40.009014]",0.000000,36.6900,0
1,1,2013-10-08T17:46:45Z,254,0,"[116.315102,40.004784]",0.562623,24.5375,0
2,2,2013-10-08T17:47:39Z,254,0,"[116.315018,40.002842]",0.778695,31.9675,0
3,3,2013-10-08T17:49:26Z,254,0,"[116.315041,39.998585]",1.252148,19.7850,0
4,4,2013-10-08T17:51:15Z,254,0,"[116.315605,39.992554]",1.924533,24.4500,0
...,...,...,...,...,...,...,...,...
414839,518403,2013-10-30T23:14:35Z,4954,21998,"[116.333366,39.79073]",21.419211,27.1450,0
414840,518404,2013-10-30T23:14:59Z,4954,21998,"[116.330444,39.79084]",21.669183,30.0825,0
414841,518405,2013-10-30T23:16:35Z,4954,21998,"[116.328453,39.793888]",22.129037,46.0550,0
414842,518406,2013-10-30T23:16:59Z,4954,21998,"[116.328072,39.796574]",22.429494,51.5150,0


In [3]:
from geopy.distance import geodesic
from math import *
"""
    计算两个经纬度坐标之间的距离
    :param coord1, coord2: 第一个坐标，格式为 (经度1, 纬度1), (经度2, 纬度2)
    :return: 距离（单位：米）
"""
def calculate_distance(coord1, coord2):
    coord_1 = (coord1[1], coord1[0])
    coord_2 = (coord2[1], coord2[0])
    distance = geodesic(coord_1, coord_2).meters
    return distance

calculate_distance((116.318726,40.009014), (116.315102,40.004784))

562.4475920054873

In [4]:
traj_ori['time'] = pd.to_datetime(traj_ori['time'])

In [5]:
traj_ori.loc[1,'time'] - traj_ori.loc[0,'time']

Timedelta('0 days 00:01:45')

In [6]:
traj_ori['time_diff'] = traj_ori['time'].diff()
traj_ori['dis_diff'] = traj_ori['current_dis'].diff() * 1000

In [7]:
traj_ori

,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays,time_diff,dis_diff
0,0,2013-10-08 17:45:00+00:00,254,0,"[116.318726,40.009014]",0.000000,36.6900,0,NaT,NaN
1,1,2013-10-08 17:46:45+00:00,254,0,"[116.315102,40.004784]",0.562623,24.5375,0,0 days 00:01:45,562.623059
2,2,2013-10-08 17:47:39+00:00,254,0,"[116.315018,40.002842]",0.778695,31.9675,0,0 days 00:00:54,216.072372
3,3,2013-10-08 17:49:26+00:00,254,0,"[116.315041,39.998585]",1.252148,19.7850,0,0 days 00:01:47,473.452423
4,4,2013-10-08 17:51:15+00:00,254,0,"[116.315605,39.992554]",1.924533,24.4500,0,0 days 00:01:49,672.385605
...,...,...,...,...,...,...,...,...,...,...
414839,518403,2013-10-30 23:14:35+00:00,4954,21998,"[116.333366,39.79073]",21.419211,27.1450,0,0 days 00:00:36,209.337371
414840,518404,2013-10-30 23:14:59+00:00,4954,21998,"[116.330444,39.79084]",21.669183,30.0825,0,0 days 00:00:24,249.972523
414841,518405,2013-10-30 23:16:35+00:00,4954,21998,"[116.328453,39.793888]",22.129037,46.0550,0,0 days 00:01:36,459.854339
414842,518406,2013-10-30 23:16:59+00:00,4954,21998,"[116.328072,39.796574]",22.429494,51.5150,0,0 days 00:00:24,300.456659


这里的cost_time是分钟，对应dis_diff的距离

In [8]:
traj_ori['traj_cost_time'] = traj_ori['time_diff'].dt.total_seconds() / 60
traj_ori

,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays,time_diff,dis_diff,traj_cost_time
0,0,2013-10-08 17:45:00+00:00,254,0,"[116.318726,40.009014]",0.000000,36.6900,0,NaT,NaN,NaN
1,1,2013-10-08 17:46:45+00:00,254,0,"[116.315102,40.004784]",0.562623,24.5375,0,0 days 00:01:45,562.623059,1.750000
2,2,2013-10-08 17:47:39+00:00,254,0,"[116.315018,40.002842]",0.778695,31.9675,0,0 days 00:00:54,216.072372,0.900000
3,3,2013-10-08 17:49:26+00:00,254,0,"[116.315041,39.998585]",1.252148,19.7850,0,0 days 00:01:47,473.452423,1.783333
4,4,2013-10-08 17:51:15+00:00,254,0,"[116.315605,39.992554]",1.924533,24.4500,0,0 days 00:01:49,672.385605,1.816667
...,...,...,...,...,...,...,...,...,...,...,...
414839,518403,2013-10-30 23:14:35+00:00,4954,21998,"[116.333366,39.79073]",21.419211,27.1450,0,0 days 00:00:36,209.337371,0.600000
414840,518404,2013-10-30 23:14:59+00:00,4954,21998,"[116.330444,39.79084]",21.669183,30.0825,0,0 days 00:00:24,249.972523,0.400000
414841,518405,2013-10-30 23:16:35+00:00,4954,21998,"[116.328453,39.793888]",22.129037,46.0550,0,0 days 00:01:36,459.854339,1.600000
414842,518406,2013-10-30 23:16:59+00:00,4954,21998,"[116.328072,39.796574]",22.429494,51.5150,0,0 days 00:00:24,300.456659,0.400000


In [9]:
traj_ori.loc[1,'traj_id']

0

In [16]:
# 按照traj_id对数据进行分组
grouped = traj_ori.groupby('traj_id')

# 使用apply函数删除每个分组的最后一行
df = grouped.apply(lambda x: x[:-1])

# 重置索引
df = df.reset_index(drop=True)

In [17]:
df

,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays,time_diff,dis_diff,traj_cost_time
0,0,2013-10-08 17:45:00+00:00,254,0,"[116.318726,40.009014]",0.000000,36.6900,0,NaT,NaN,NaN
1,1,2013-10-08 17:46:45+00:00,254,0,"[116.315102,40.004784]",0.562623,24.5375,0,0 days 00:01:45,562.623059,1.750000
2,2,2013-10-08 17:47:39+00:00,254,0,"[116.315018,40.002842]",0.778695,31.9675,0,0 days 00:00:54,216.072372,0.900000
3,3,2013-10-08 17:49:26+00:00,254,0,"[116.315041,39.998585]",1.252148,19.7850,0,0 days 00:01:47,473.452423,1.783333
4,4,2013-10-08 17:51:15+00:00,254,0,"[116.315605,39.992554]",1.924533,24.4500,0,0 days 00:01:49,672.385605,1.816667
...,...,...,...,...,...,...,...,...,...,...,...
397238,518402,2013-10-30 23:13:59+00:00,4954,21998,"[116.335815,39.79068]",21.209873,41.3425,0,0 days 00:00:52,305.173403,0.866667
397239,518403,2013-10-30 23:14:35+00:00,4954,21998,"[116.333366,39.79073]",21.419211,27.1450,0,0 days 00:00:36,209.337371,0.600000
397240,518404,2013-10-30 23:14:59+00:00,4954,21998,"[116.330444,39.79084]",21.669183,30.0825,0,0 days 00:00:24,249.972523,0.400000
397241,518405,2013-10-30 23:16:35+00:00,4954,21998,"[116.328453,39.793888]",22.129037,46.0550,0,0 days 00:01:36,459.854339,1.600000


In [18]:
df.to_csv('traj_mid.csv',index=False)

In [15]:
index = 0
traj_ori['traj_id']
for i in range(len(traj_ori)):
    # print(traj_ori.loc[i,'traj_id'])
    if(traj_ori.loc[i,'traj_id'] != index):
        traj_ori = traj_ori.drop(i - 1)
        index = traj_ori.loc[i,'traj_id']
traj_ori

KeyError: 21

In [ ]:
traj_ori['holidays'].value_counts()